<a href="https://colab.research.google.com/github/00150/AI_07_JYP_Project_section_02/blob/main/%F0%9F%8C%8F_%EC%9A%B0%ED%81%AC%EB%9D%BC%EC%9D%B4%EB%82%98_vs_%EB%9F%AC%EC%8B%9C%EC%95%84_%EC%A0%84%EC%9F%81_%EB%B6%84%EC%84%9D_(feat_%EB%94%94%ED%85%8C%EC%9D%BC%ED%95%98%EA%B2%8C_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🌏 우크라이나 vs 러시아 전쟁 분석 (feat.디테일하게..)
> 지난번 우리는 우크라이나 러시아 전쟁에 대하여 다중선형회귀까지 진행했습니다.
하지만, 데이터의 이상치 및 편향과 분산등을 전혀 고려하지 않은 채 진행하였습니다.

* 데이터의 이상치 및 정확한 분석에 해가 될 수 있는 부분을 제거하여 최적의 피처를 선택하여 분석히여 봅시다.
* 이번 분석 또한 훈련 및 테스트 데이터를 나눌 때 '2022-08-08'을 기준으로 나눠보겠습니다.

✅ 참고 : 파이썬 내장 함수에서 timestamp에 대하여 float() 등으로 자동으로 변환해주지 않습니다. </br>
- 최적의 피처를 찾는 SelectKBest를 사용하는 경우 시계열 데이터 때문에 'TypeError' 가 발생할 수 있으므로,</br> 
지금 당장은 변환하지 않겠습니다. 

In [ ]:
# 데이터 가져오기.
import pandas as pd
import numpy as np


# 데이터 주소
url_01 = '/content/drive/MyDrive/new_project02/최근_러시아_인명피해_자료.csv'
url_02 = '/content/drive/MyDrive/new_project02/최근_러시아_장비손실_자료.csv'


#데이터 불러오기
df_01 = pd.read_csv(url_01)
df_02 = pd.read_csv(url_02)


# 병합
df = pd.merge(df_01, df_02, on =['date', 'day'])
# 165번 행부터 진행할 것 입니다. 또한 '사망자_수' 나타내는 컬럼은 삭제합니다.

# 컬럼명 변경
df = df.rename(columns = {'date':'날짜','day':'개전일','aircraft':'항공기','helicopter':'헬리콥터','tank' : '탱크',
                    'APC':'장갑차','field artillery': '야포' ,'MRL' :'다중로켓발사기','military auto' : '자동화기기',
                    'fuel tank': '연료탱크', 'drone':'드론' ,'naval ship' : '해군함정', 'anti-aircraft warfare' : '방공포', 'special equipment' : '특수_장비',
                    'mobile SRBM system' : '자동화_탄도_미사일', 'vehicles and fuel tanks' : '탈것_및_연료탱크', 'cruise missiles' : '크루즈_미사일',
                    'greatest losses direction' : '피해가_큰_지역','personnel':'사망자_수', 'personnel*':'불확실한_사망자', 'POW':'전쟁_포로'})


# 컬럼: 날짜 datetime 형태로 변경
#df['날짜'] = pd.to_datetime(df['날짜'])


# 누적데이터 -> 일별 데이터로 변경
# 각 컬럼별 차를 계산할 수 없는 데이터 인덱스로 내리기.
df = df.set_index(['날짜', '개전일', '피해가_큰_지역','불확실한_사망자'])

# 각 컬럼별 차이 계산 및 결측값 채우기
df = df.diff().fillna(df).fillna(0).astype(int).reset_index()

# 컬럼 : '피해가_큰_지역' 에 대하여 결측값 채워넣기.
df['피해가_큰_지역'] = df['피해가_큰_지역'].fillna('unknown area')

# 데이터 내의 결측값 확인하기.
# df.isnull().sum() -> 모든 컬럼에 대하여 결측값 : 0


# '2022-08-08'을 기준으로 훈련/테스트 세트를 분리합니다.
# '2022-08-08'은 훈련 데이터에 포함되어야 합니다.
cutoff = pd.to_datetime('2022-08-08')
train = df[df['날짜'] <= cutoff]
test = df[df['날짜'] > cutoff] 

# --------------- X_train, y_train, X_test, y_test로 데이터 나누기, 

# target = {column : '사망자_수'}
target = '사망자_수'

# X_train
X_train = train.drop(columns=target)

# y_train
y_train = train[target]

# X_test
X_test = test.drop(columns=target)

# y_test
y_test = test[target]


# 데이터 분할 확인하기.
print(f'훈련(독립) : {X_train.shape}, 훈련(종속) :{y_train.shape} \n테스트(독립){X_test.shape}, 테스트(종속){y_test.shape}')

훈련(독립) : (165, 20), 훈련(종속) :(165,) 
테스트(독립)(26, 20), 테스트(종속)(26,)


데이터 분할을 모두 완료하였습니다 😗

### Part 1. 정확도를 올려 데이터를 분석합시다.
> 다양한 모델을 만들었다고 하더라도, 정확도가 좋지 못하면 '쓰레기'입니다 😥

❗ SelectKBest를 사용하여 가장 효과적인 특성 $K$개를 골라봅시다.</br>
❗ 우리의 목표는 target과 가장 관련 있는 features 를 $K$개 고르는 것이 목표입니다.


In [ ]:
# 데이터의 feature 개수 구하기. -> 훈련 및 테스트 데이터에 대한 종속변수의 피처개수는 동일하므로, 훈련데이터(종속)을 기준으로 정리하였다.
X_train.columns.value_counts().sum()

20

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165 entries, 0 to 164
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   날짜          165 non-null    datetime64[ns]
 1   개전일         165 non-null    int64         
 2   피해가_큰_지역    165 non-null    object        
 3   불확실한_사망자    165 non-null    object        
 4   전쟁_포로       165 non-null    int64         
 5   항공기         165 non-null    int64         
 6   헬리콥터        165 non-null    int64         
 7   탱크          165 non-null    int64         
 8   장갑차         165 non-null    int64         
 9   야포          165 non-null    int64         
 10  다중로켓발사기     165 non-null    int64         
 11  자동화기기       165 non-null    int64         
 12  연료탱크        165 non-null    int64         
 13  드론          165 non-null    int64         
 14  해군함정        165 non-null    int64         
 15  방공포         165 non-null    int64         
 16  특수_장비       165 non-null  

In [ ]:
#f_regression : 회귀작업에 대한 레이블/특징사이의 F값.  * 좀 더 찾아봐서 정리할 부분.
from sklearn.feature_selection import f_regression, SelectKBest

# Selector 정의. 
# 종속변수를 담은  데이터의 feature 개수가 총 20개이므로, 10개만 찾아서 사용하여봅시다.
selector = SelectKBest(score_func = f_regression, k = 10)


# 학습데이터 -> fit_transform
X_train_selected = selector.fit_transform(X_train, y_train)


# 테스트데이터 -> transform
X_test_selected = transform(X_test)






TypeError: ignored